<h1>Data Streaming Algorithms</h1>

<h2>HW1</h2>

Slava Salasin, 319238119
TODO - Bayan

<h2>Dry Part

<h3>Morris algorithm

In order to

<h3>Reservoir sampling

In class we proved the Reservoir sampling algorithm for $j>k$. Now we will provide a supplementary proof for $j<=k$.
<br>
$P[a_{j}\ will\ be\ sampled] =_{1} P[a_{j}\ will\ not\ be\ replaced\ on\ j=k+1] \cdot  ... \cdot P[a_{j}\ will\ not\ be\ replaced\ on\ j=n] =_{2} (1 - P[a_{j}\ will\ be\ replaced\ on\ j=k+1]) \cdot  ... \cdot (1 - P[a_{j}\ will\ be\ replaced\ on\ j=n]) =_{3} (1 - \dfrac{1}{k+1}) \cdot  ... \cdot (1 - \dfrac{1}{n}) =_{4} \dfrac{k}{k+1} \cdot  ... \cdot \dfrac{n-1}{n} =_{5} \dfrac{k}{n}$
<br>
1. If $a_{j}$ is sampled, then it's not replaced in iteration k+1 and k+2 etc.
2. Probability complement.
3. $P[a_{j}\ will\ be\ replaced\ on\ j=k+1] = \dfrac{k}{k+1} \cdot \dfrac{1}{k}$

    $\dfrac{k}{k+1}$ - The probability to sample an element from the stream is the sample size divided by the element index. $\dfrac{1}{k}$ - The probabiliyy to replace a specific element from the sample is 1 divided by the size of the sample.

4. Simple algebra.
5. Simple algebra.

In [243]:
import numpy as np

np.random.seed(42)

In [ ]:
def simulate(experiment_func, experiment_func_input):
    results = []
    for simulation in range(50):
        results.append(experiment_func(experiment_func_input))
    return int(np.mean(results))

In [244]:
def fm_alpha(stream_arg):
    hash_function = dict(map(lambda key, value : (key, value) , range(1, 10001), np.random.uniform(0, 1, 10000)))
    x = 1
    for s in stream_arg:
        h_a = hash_function.get(s)
        if h_a < x:
            x = h_a
    return int(1 / x), x

In [245]:
def fm_beta(stream_arg, instances_arg = None):
    x_s = []
    for _ in range(instances_arg):
        _, min_hash = fm_alpha(stream_arg)
        x_s.append(min_hash)
    return int((1 / np.mean(x_s)) - 1)

In [246]:
def fm_final():
    pass

In [247]:
stream = []
for i in range(1, 10001):
    stream.extend(i for j in range(np.random.randint(10, 50 + 1)))
np.random.shuffle(stream)
unique, counts = np.unique(stream, return_counts=True)
print(f'unique elements in stream {len(unique)}')
print(f'total elements in stream {sum(counts)}')
uniques, _ = fm_alpha(stream)
print(f'FM alpha unique elements estimation {uniques}')

unique elements in stream 10000
total elements in stream 300172
FM alpha unique elements estimation 435126


In [248]:
for instances in [10, 25, 50]:

    print(f'FM beta with {instances} instances yields {fm_beta(stream, instances)} unique elements in the stream')

FM beta with 10 instances yields 13619 unique elements in the stream
FM beta with 25 instances yields 12565 unique elements in the stream
FM beta with 50 instances yields 8587 unique elements in the stream
